# Bachelor Thesis

## LoRa Time on Air

In [10]:
class LoraConf:
    preamble_len = 12
    crc = True
    sf = 7 # [7..12]
    cr = 1 # [0..4]: coding-rate is calculated as follows: 4/(4+cr)
    bw = 125_000
    def __init__(self, preamble_len = 12, crc = True, sf = 7, cr = 1, bw = 125_000):
        self.preamble_len = preamble_len
        self.crc = crc
        self.sf = sf
        self.cr = cr
        self.bw = bw

lora_conf = LoraConf(8, True, 7, 1, 125_000)

Implement time on air (ToA) calculation as described in [STM32WL5X reference manual](https://www.st.com/content/ccc/resource/technical/document/reference_manual/group0/34/ea/c6/91/75/6f/45/27/DM00451556/files/DM00451556.pdf/jcr:content/translations/en.DM00451556.pdf)

In [11]:
def ceil(frac, base):
    return ((frac // base) + 1 ) * base

def lora_time_on_air(lora_conf, user_payload_bytes):
    nb_payload_symbols_frac = ((user_payload_bytes * 8 + int(lora_conf.crc) * 16 - 4 * (lora_conf.sf - 7)) * (4 + lora_conf.cr)) / (4 * lora_conf.sf)
    nb_payload_symbols = ceil(nb_payload_symbols_frac, lora_conf.cr + 4)
    return (lora_conf.preamble_len + nb_payload_symbols + 4.25 + 8) * (2**lora_conf.sf / lora_conf.bw)

### ToA calculations

#### Ping_t
```c
typedef struct {
    uint8_t device_id;
    uint_t packet_id;
} Ping_t;
```
#### AnchorResponse_t
```c
typedef struct {
    char anchor_id;
    int16_t recv_rssi;
} AnchorResponse_t;
```
#### Ack_t
```c
typedef struct {
    char anchor_id;
    uint8_t packet_id;
} Ack_t;
```

In [12]:
print(f"ToA(Ping_t) = {lora_time_on_air(lora_conf, 2) * 1000}ms")
print(f"ToA(AnchorResponse_t) = {lora_time_on_air(lora_conf, 3) * 1000}ms")
print(f"ToA(Ack_t) = {lora_time_on_air(lora_conf, 2) * 1000}ms")

ToA(Ping_t) = 30.976ms
ToA(AnchorResponse_t) = 30.976ms
ToA(Ack_t) = 30.976ms
